In [ ]:
pip install transformers


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
reviews= pd.read_csv(r'/content/sample_data/reviews_vader_sentiment.csv',index_col=0)

reviews.head()

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Recommended,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment
0,This has been a total nightmare,1,V Darkana,1.0,My flight was scheduled on July 4 2021 Sunda...,Family Leisure,Economy Class,Anchorage to San Francisco,Jul-21,1.0,...,no,3.0,3.0,1.0,United Airlines,2021,July,0.0,International,negative
1,beyond appalled by this experience,1,Bishop Jermaine Moore Sr.,0.0,United Airlines have successfully and repetit...,Family Leisure,Economy Class,Pittsburgh to Dubai via Newark,May-23,3.0,...,no,1.0,1.0,1.0,United Airlines,2023,May,1.0,International,negative
2,Both segments were awful,1,J Parcell,1.0,Both segments were awful. The single agent f...,Solo Leisure,Economy Class,Montrose to Denver,Jul-21,2.0,...,no,1.0,1.0,1.0,United Airlines,2021,July,0.0,International,negative
3,never fly through this airline again,1,S Laytan,1.0,We reviewed the dimensions and the sizes on ...,Family Leisure,Economy Class,Pasco to Miami,Jun-21,1.0,...,no,1.0,1.0,1.0,United Airlines,2021,June,0.0,International,positive
4,fly other airlines in the future,1,Nancy S Giannetta,1.0,So as I boarded I ask for help putting my ca...,Solo Leisure,Business Class,Chicago to Tampa,Apr-24,2.0,...,no,2.0,2.0,1.0,United Airlines,2024,April,0.0,International,positive


In [ ]:
import spacy

In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
# Remove "not" and words ending with "n't" from stop words
custom_stopwords = set()
for word in nlp.Defaults.stop_words:
    if word == "not" or word.endswith("n't"):
        custom_stopwords.add(word)

# Remove these custom stop words from the spaCy stop word list
for word in custom_stopwords:
    nlp.vocab[word].is_stop = False


In [ ]:
nlp.vocab["aren't"].is_stop

False

In [ ]:
reviews['Airline_Name'].value_counts()

,count
Airline_Name,
American Airlines,1901
Spirit Airlines,1815
Frontier Airlines,1799
United Airlines,1425
Delta Airlines,1246
Jet Blue Airlines,844
South West Airlines,774
Allegiant Air Airlines,705


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Example review



vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
def preprocess(text):
    text=nlp(text)
    filtered_text=[]
    for each in text:
        if each.is_stop or each.is_punct:
          continue

        else:
            filtered_text.append(each.lemma_)
    return ' '.join(filtered_text)

In [ ]:
preprocess('kumar is not working today')

'kumar not work today'

In [ ]:
def sentiment_score(text):
  review = preprocess(text)
# Tokenize and encode the review text
  inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True,max_length=512)

# Perform inference (no gradient calculation needed)
  with torch.no_grad():
      outputs = model(**inputs)
      predictions = torch.softmax(outputs.logits, dim=-1)

  # Extract the predicted sentiment label
  predicted_class = predictions.argmax().item()
  label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

  return label_mapping[predicted_class]


In [ ]:
reviews.shape

(10509, 23)

In [ ]:
reviews['Experience_Detail'].iloc[0:3000]

,Experience_Detail
0,My flight was scheduled on July 4 2021 Sunda...
1,United Airlines have successfully and repetit...
2,Both segments were awful. The single agent f...
3,We reviewed the dimensions and the sizes on ...
4,So as I boarded I ask for help putting my ca...
...,...
2995,Please don't fly on this airline. Unless you...
2996,My family's flight was canceled an hour befor...
2997,I experienced significant challenges with Am...
2998,What a terrible experience. We were kept on ...


In [ ]:
reviews_1=reviews.iloc[0:3000]
reviews_2=reviews.iloc[3000:6000]
reviews_3=reviews.iloc[6000:9000]
reviews_4=reviews.iloc[9000:10510]
#reviews['roberta_sentiment']= reviews['Experience_Detail'].iloc[2:4].apply(sentiment_score)

In [ ]:
reviews_1['roberta_sentiment']= reviews_1['Experience_Detail'].apply(sentiment_score)

<ipython-input-29-4eba50179dbb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_1['roberta_sentiment']= reviews_1['Experience_Detail'].apply(sentiment_score)


In [ ]:
reviews_1.iloc[2998:2999]

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment,roberta_sentiment
2998,What a terrible experience,1,Anthony Rizzi,0.0,What a terrible experience. We were kept on ...,Solo Leisure,Economy Class,La Guardia to Washington,Jul-22,1.0,...,1.0,1.0,1.0,American Airlines,2022,July,0.0,International,neutral,Negative


In [ ]:
reviews_2['roberta_sentiment']= reviews_2['Experience_Detail'].apply(sentiment_score)

<ipython-input-31-c6a07b545bd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_2['roberta_sentiment']= reviews_2['Experience_Detail'].apply(sentiment_score)


In [ ]:
reviews_2

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment,roberta_sentiment
3000,Expect your flight to be changed,3,A Darlen,1.0,Expect your flight to be changed 70% of the ...,Solo Leisure,Economy Class,Buenos Aires to Mobile via Dallas,Jul-22,2.0,...,5.0,5.0,1.0,American Airlines,2022,July,1.0,International,positive,Neutral
3001,on the phone for 5 hours with the baggage claim,1,Roxie Daniels,0.0,Was on the phone for 5 hours with the baggage...,Solo Leisure,Economy Class,Norfolk to Austin,Jan-22,3.0,...,1.0,1.0,1.0,American Airlines,2022,January,0.0,International,negative,Neutral
3002,need to have better customer service training,1,Jeff Hepburn,0.0,"Upon boarding the plane, the staff scanning ...",Family Leisure,Economy Class,Dallas to Jackson,Dec-23,2.0,...,2.0,2.0,2.0,American Airlines,2023,December,0.0,International,neutral,Neutral
3003,this is just absurd,1,Jessica Harmon,0.0,Apparently due to COVID-19 they are just ste...,Couple Leisure,Economy Class,"San Diego, CA to Vancouver, Canada",Dec-20,1.0,...,1.0,1.0,1.0,American Airlines,2020,December,0.0,International,negative,Negative
3004,canceled my flight and booked with Delta,1,R Baynes,1.0,This was the worst experience I ever had at ...,Business,Economy Class,Charleston to Washington,Jul-23,2.0,...,2.0,2.0,2.0,American Airlines,2023,July,0.0,International,negative,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,experience was extremely disappointing,3,Kenneth Trestka,0.0,"The booking process went smoothly. However, ...",Business,Economy Class,New York to Las Vegas,May-23,2.0,...,2.0,2.0,2.0,Jet Blue Airlines,2023,May,0.0,International,negative,Negative
5996,Will never use JetBlue again,1,Doron Matmor,1.0,Avoid. Flight schedule at 5pm. On the way to...,Solo Leisure,Economy Class,Jacksonville to Boston,Jun-23,1.0,...,3.0,1.0,2.0,Jet Blue Airlines,2023,June,0.0,International,negative,Negative
5997,worst customer service,1,D Larego,1.0,The worst customer service I have experience...,Family Leisure,Economy Class,New York to Quito via Ft Lauderdale,Jun-23,1.0,...,1.0,1.0,1.0,Jet Blue Airlines,2023,June,1.0,International,negative,Negative
5998,I will never fly JetBlue again,1,Danielle Hauck,0.0,Haven't even flown with them yet and it's al...,Solo Leisure,Economy Class,San Jose to Miami,Jun-23,1.0,...,1.0,1.0,1.0,Jet Blue Airlines,2023,June,0.0,International,positive,Neutral


In [ ]:
reviews_3['roberta_sentiment']= reviews_3['Experience_Detail'].apply(sentiment_score)

<ipython-input-35-42e3fdef0ec5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_3['roberta_sentiment']= reviews_3['Experience_Detail'].apply(sentiment_score)


In [ ]:
reviews_4['roberta_sentiment']= reviews_4['Experience_Detail'].apply(sentiment_score)

<ipython-input-36-b6bde2e76685>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_4['roberta_sentiment']= reviews_4['Experience_Detail'].apply(sentiment_score)


In [ ]:
reviews.head()

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment,roberta_sentiment
0,This has been a total nightmare,1,V Darkana,1.0,My flight was scheduled on July 4 2021 Sunda...,Family Leisure,Economy Class,Anchorage to San Francisco,Jul-21,1.0,...,3.0,3.0,1.0,United Airlines,2021,July,0.0,International,negative,NaN
1,beyond appalled by this experience,1,Bishop Jermaine Moore Sr.,0.0,United Airlines have successfully and repetit...,Family Leisure,Economy Class,Pittsburgh to Dubai via Newark,May-23,3.0,...,1.0,1.0,1.0,United Airlines,2023,May,1.0,International,negative,NaN
2,Both segments were awful,1,J Parcell,1.0,Both segments were awful. The single agent f...,Solo Leisure,Economy Class,Montrose to Denver,Jul-21,2.0,...,1.0,1.0,1.0,United Airlines,2021,July,0.0,International,negative,Negative
3,never fly through this airline again,1,S Laytan,1.0,We reviewed the dimensions and the sizes on ...,Family Leisure,Economy Class,Pasco to Miami,Jun-21,1.0,...,1.0,1.0,1.0,United Airlines,2021,June,0.0,International,positive,Neutral
4,fly other airlines in the future,1,Nancy S Giannetta,1.0,So as I boarded I ask for help putting my ca...,Solo Leisure,Business Class,Chicago to Tampa,Apr-24,2.0,...,2.0,2.0,1.0,United Airlines,2024,April,0.0,International,positive,NaN


In [ ]:
reviews_combined= pd.concat([reviews_1,reviews_2,reviews_3,reviews_4],axis=0)

In [ ]:
reviews_combined.shape

(10509, 24)

In [ ]:
reviews_combined.head()

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment,roberta_sentiment
0,This has been a total nightmare,1,V Darkana,1.0,My flight was scheduled on July 4 2021 Sunda...,Family Leisure,Economy Class,Anchorage to San Francisco,Jul-21,1.0,...,3.0,3.0,1.0,United Airlines,2021,July,0.0,International,negative,Negative
1,beyond appalled by this experience,1,Bishop Jermaine Moore Sr.,0.0,United Airlines have successfully and repetit...,Family Leisure,Economy Class,Pittsburgh to Dubai via Newark,May-23,3.0,...,1.0,1.0,1.0,United Airlines,2023,May,1.0,International,negative,Negative
2,Both segments were awful,1,J Parcell,1.0,Both segments were awful. The single agent f...,Solo Leisure,Economy Class,Montrose to Denver,Jul-21,2.0,...,1.0,1.0,1.0,United Airlines,2021,July,0.0,International,negative,Negative
3,never fly through this airline again,1,S Laytan,1.0,We reviewed the dimensions and the sizes on ...,Family Leisure,Economy Class,Pasco to Miami,Jun-21,1.0,...,1.0,1.0,1.0,United Airlines,2021,June,0.0,International,positive,Neutral
4,fly other airlines in the future,1,Nancy S Giannetta,1.0,So as I boarded I ask for help putting my ca...,Solo Leisure,Business Class,Chicago to Tampa,Apr-24,2.0,...,2.0,2.0,1.0,United Airlines,2024,April,0.0,International,positive,Neutral


In [ ]:
reviews_combined.columns

Index(['Experience_Summary', 'Overall_Rating', 'Customer_Name',
       'Trip_Verified', 'Experience_Detail', 'Traveller_Type', 'Seat_Type',
       'Flight_Route', 'Flight_Date', 'Seat_Comfort', 'Cabin_Staff_Service',
       'Ground_Service', 'Value_for_Money', 'Recommended',
       'Food_and_Beverages', 'Inflight_Entertainment', 'WiFi_and_Connectivity',
       'Airline_Name', 'Year_Flown', 'Month_Flown', 'Number_of_Stops',
       'Trip_Type', 'vader_sentiment', 'roberta_sentiment'],
      dtype='object')

In [ ]:
reviews_combined['roberta_sentiment'].value_counts()

,count
roberta_sentiment,
Negative,7198
Neutral,2553
Positive,758


In [ ]:
reviews_combined['vader_sentiment'].value_counts()

,count
vader_sentiment,
negative,6273
positive,2515
neutral,1721


In [ ]:
reviews_combined['Recommended'].value_counts()

,count
Recommended,
no,9412
yes,1097


In [ ]:
reviews_combined.to_csv('reviews_with_sentiments.csv')